In [7]:
%load_ext autoreload
%autoreload 2
import time
import numpy as np
from iblatlas.regions import BrainRegions
from one.api import ONE
one = ONE(base_url='https://alyx.internationalbrainlab.org')

import iblatlas.atlas as atlas
import pickle
import pandas as pd
import unityneuro.render as urn
import seaborn as sns
"""
Display all trajectories in one single brain volume
Author: Mayo, Gaelle, Dan
"""

import numpy as np



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import paper-reproducible-ephys.reproducible_ephys_functions as re_funcs

SyntaxError: invalid syntax (3121415701.py, line 1)

In [3]:
urn.setup()

(URN) connected to server
Login sent with ID: Dan


In [4]:
(lab_number_map, institution_map, institution_colors) = re_funcs.labs()

NameError: name 're_funcs' is not defined

# Export all histology alignments

In [6]:
bwm_trajs = one.alyx.rest('trajectories', 'list', provenance='Ephys aligned histology track',
                                   project='ibl_neuropixel_brainwide_01', use_cache=False)

In [52]:
traj_data = pd.DataFrame(columns=['pid','lab','ml','ap','dv','phi','theta','depth'])
for i,traj in enumerate(bwm_trajs):
    ins = atlas.Insertion.from_dict(traj)

    mlapdv = np.round(ba.xyz2ccf(ins.tip))

    traj_data.loc[i] = [traj['probe_insertion'],traj['session']['lab'],mlapdv[0],mlapdv[1],mlapdv[2],traj['phi'],traj['theta'],traj['depth']]

In [57]:
traj_data.to_csv('bwm_trajectory_data.csv',float_format='%.03f')

In [59]:
np.unique(traj_data['lab'].values)

array(['angelakilab', 'churchlandlab', 'churchlandlab_ucla', 'cortexlab',
       'danlab', 'hoferlab', 'mainenlab', 'mrsicflogellab',
       'steinmetzlab', 'wittenlab', 'zadorlab'], dtype=object)

# Reproducible ephys trajectories

In [8]:
trajectories = one.alyx.rest('trajectories', 'list', provenance='Planned', x=-2243, y=-2000, theta=15,
                                   project='ibl_neuropixel_brainwide_01', use_cache=False)

In [9]:
def getCoords(ins):
    entry_coords = ba.xyz2ccf(ins.entry)
    tip_coords = ba.xyz2ccf(ins.tip)
    tip_angles = [ins.phi, ins.theta, ins.beta]
    recording_size = np.sqrt(np.sum(np.power(entry_coords-tip_coords,2)))
    return (tip_coords, tip_angles, recording_size)

In [ ]:
trajectories


In [24]:
#This block is only needed if you are re-running this from scratch, otherwise just load the .pkl file

# Pull the micro-manip and histology tracks for each of these recordings
data = {}



for traj in trajectories:
    pid = traj['probe_insertion']
    
    
    traj_histology = one.alyx.rest('trajectories', 'list',
        provenance='Ephys aligned histology track',
        probe_insertion=pid, use_cache=False)

    if len(traj_histology) == 0:
        traj_histology = one.alyx.rest('trajectories', 'list', provenance='Histology track',
            probe_insertion=pid, use_cache=False)
        
    traj_micro = one.alyx.rest('trajectories', 'list',
        provenance='Micro-manipulator',
        probe_insertion=pid, use_cahce=False)

    # skip any sessions that we are missing histology for
    if len(traj_histology) == 0:
        continue

    if not traj_histology[0]['x']:
        continue

    ins_planned = atlas.Insertion.from_dict(traj)
    ins_micro = atlas.Insertion.from_dict(traj_histology[0])
    ins_histology = atlas.Insertion.from_dict(traj_micro[0])
    
    

    data[pid] = {}
    data[pid]['planned'] = getCoords(ins_planned)
    data[pid]['micro-manip'] = getCoords(ins_micro)
    data[pid]['histology'] = getCoords(ins_histology)
    
pickle.dump(data,open( './ibl_repro_ephys_traj.pkl', "wb" ))

In [12]:
data = pickle.load( open( "./ibl_repro_ephys_traj.pkl", "rb" ) )


In [14]:
data

{'1e176f17-d00f-49bb-87ff-26d237b525f1': {'planned': (array([4531.27618041, 7400.        , 4364.70330516]),
   [180.0, 15.0, 0],
   4000.000000000001),
  'micro-manip': (array([4068.08157863, 7879.71064526, 4926.00114312]),
   [137.924499592581, 13.9475790294517, 0],
   4508.9397515214705),
  'histology': (array([4531.27618041, 7400.        , 4364.70330516]),
   [180.0, 15.0, 0],
   4000.000000000001)},
 'f936a701-5f8a-4aa1-b7a9-9f8b5b69bc7c': {'planned': (array([4531.27618041, 7400.        , 4364.70330516]),
   [180.0, 15.0, 0],
   4000.000000000001),
  'micro-manip': (array([4115.31870497, 7732.89207886, 3945.59142813]),
   [174.186320692381, 9.02847198662822, 0],
   3564.7570509022507),
  'histology': (array([4450.14810497, 7419.8       , 4365.25740892]),
   [180.0, 15.0, 0],
   3702.0)},
 'f86e9571-63ff-4116-9c40-aa44d57d2da9': {'planned': (array([4531.27618041, 7400.        , 4364.70330516]),
   [180.0, 15.0, 0],
   4000.000000000001),
  'micro-manip': (array([4453.2951295 , 8122.

In [13]:
import random
r = lambda: random.randint(0,255)
randHexColor = lambda : '#%02X%02X%02X' % (r(),r(),r())

In [30]:
# VIEW 1: ephys aligned histology tracks
urn.clear()
count = 0

vdata = {'VISa': True, 'CA1': True, 'DG': True, 'LP': True, 'PO': True}
urn.set_area_visibility(vdata)
vdata = {'VISa': 'transparent-lit', 'CA1': 'transparent-lit', 'DG': 'transparent-lit', 'LP': 'transparent-lit', 'PO': 'transparent-lit'}
urn.set_area_material(vdata)
vdata = {'VISa': 0.4, 'CA1': 0.4, 'DG': 0.4, 'LP': 0.4, 'PO': 0.4}
urn.set_area_alpha(vdata)

urn.set_area_visibility({'grey':True})
urn.set_area_material({'grey':'transparent-unlit'})
urn.set_area_color({'grey':'#000000'})
urn.set_area_alpha({'grey':0.025})

urn.set_camera_rotation(55,0,150)


for i, (pid, trajs) in enumerate(data.items()):
    probeColor = '#808080' #randHexColor()
    for j, (traj_type, traj) in enumerate(trajs.items()):
        if traj_type == "histology":
            probename = 'p'+str(count)
            count+=1
            
            tip_coords = traj[0]
            tip_angles = traj[1]
            recording_size = traj[2]

            urn.create_probes([probename])
            urn.set_probe_positions({probename:tip_coords.tolist()})
            urn.set_probe_colors({probename:probeColor})
            urn.set_probe_angles({probename:tip_angles})
            urn.set_probe_size({probename:[0.07,recording_size/1000,0.02]})

In [14]:
# VIEW 1: Probe positions planned vs micro-manipulator
urn.clear()
count = 0

vdata = {'VISa': True, 'CA1': True, 'DG': True, 'LP': True, 'PO': True}
urn.set_area_visibility(vdata)
vdata = {'VISa': 'transparent-lit', 'CA1': 'transparent-lit', 'DG': 'transparent-lit', 'LP': 'transparent-lit', 'PO': 'transparent-lit'}
urn.set_area_material(vdata)
vdata = {'VISa': 0.4, 'CA1': 0.4, 'DG': 0.4, 'LP': 0.4, 'PO': 0.4}
urn.set_area_alpha(vdata)

for i, (pid, trajs) in enumerate(data.items()):
    probeColor = randHexColor()
    for j, (traj_type, traj) in enumerate(trajs.items()):
        if traj_type == "micro-manip":
            probename = 'p'+str(count)
            count+=1
            
            tip_coords = traj[0]
            tip_angles = traj[1]
            recording_size = traj[2]

            urn.create_probes([probename])
            urn.set_probe_positions({probename:tip_coords.tolist()})
            urn.set_probe_colors({probename:probeColor})
            urn.set_probe_angles({probename:tip_angles})
            urn.set_probe_size({probename:[0.07,recording_size/1000,0.02]})

# # for some reason 'root' doesn't work?
urn.set_area_visibility({8:True})
urn.set_area_material({8:'transparent-unlit'})
urn.set_area_color({8:'#000000'})
urn.set_area_alpha({8:0.025})

In [9]:
# VIEW 2: Probe positions micro-manip vs. histology

urn.clear()
count = 0

for i, (pid, trajs) in enumerate(data.items()):
    probeColor = randHexColor()

#     vdata = {'VISa': True, 'CA1': True, 'DG': True, 'LP': True, 'PO': True}
#     umr.set_area_visibility(vdata)
#     vdata = {'VISa': 'transparent-lit', 'CA1': 'transparent-lit', 'DG': 'transparent-lit', 'LP': 'transparent-lit', 'PO': 'transparent-lit'}
#     umr.set_area_shader(vdata)
#     vdata = {'VISa': 0.4, 'CA1': 0.4, 'DG': 0.4, 'LP': 0.4, 'PO': 0.4}
#     umr.set_area_alpha(vdata)

#     # # for some reason 'root' doesn't work?
#     umr.set_area_visibility({8:True})
#     umr.set_area_shader({8:'transparent-unlit'})
#     umr.set_area_color({8:'#000000'})
#     umr.set_area_alpha({8:0.025})

    for j, (traj_type, traj) in enumerate(trajs.items()):
        if traj_type == "planned":
            probename = 'p'+str(count)
            count+=1
            
            tip_coords = traj[0]
            tip_angles = traj[1]
            recording_size = traj[2]

            urn.create_probes([probename])
            urn.set_probe_positions({probename:tip_coords.tolist()})
            urn.set_probe_colors({probename:"#000000"})
            urn.set_probe_angles({probename:tip_angles})
            urn.set_probe_size({probename:[0.07,recording_size/1000,0.02]})
            
        if traj_type == "micro-manip":
            probename = 'p'+str(count)
            count+=1
            
            tip_coords = traj[0]
            tip_angles = traj[1]
            recording_size = traj[2]

            urn.create_probes([probename])
            urn.set_probe_positions({probename:tip_coords.tolist()})
            urn.set_probe_colors({probename:"#FF0000"})
            urn.set_probe_angles({probename:tip_angles})
            urn.set_probe_size({probename:[0.07,recording_size/1000,0.02]})
            
        if traj_type == "histology":
            probename = 'p'+str(count)
            count+=1
            
            tip_coords = traj[0]
            tip_angles = traj[1]
            recording_size = traj[2]

            urn.create_probes([probename])
            urn.set_probe_positions({probename:tip_coords.tolist()})
            urn.set_probe_colors({probename:"#00FF00"})
            urn.set_probe_angles({probename:tip_angles})
            urn.set_probe_size({probename:[0.07,recording_size/1000,0.02]})
            


(URN) disconnected from server
